In [1]:
# Import standard modules
import datetime

# Import installed modules
import numpy as np
import spiceypy

In [2]:
# Load the SPICE kernels via a meta file
spiceypy.furnsh('kernel_meta.txt')

# Set a datetime format
datetime_format = "%Y-%m-%dT%H:%M:%S"

# Create an initial date-time object that is converted to a string
init_datetime_utc = datetime.datetime(year=2021,
                                      month=11,
                                      day=19,
                                      hour=0,
                                      minute=0,
                                      second=0).strftime(datetime_format)

# Convert to Ephemeris Time (ET) using the SPICE function utc2et
datetime_et = spiceypy.utc2et(init_datetime_utc)

In [3]:
# On spaceweather.com we can see that an asteroid has a close Earth fly-by:
# 2021 WP on 2021-November-21 at a distance of around 0.4 LD, so within the Sphere Of Influence of
# Earth!

# 1 AU in km
ONE_AU = spiceypy.convrt(x=1, inunit='AU', outunit='km')

# Get the G*M value for the Sun
_, gm_sun_pre = spiceypy.bodvcd(bodyid=10, item='GM', maxn=1)
GM_SUN = gm_sun_pre[0]

# Set the G*M parameter of our planet
_, gm_earth_pre = spiceypy.bodvcd(bodyid=399, item='GM', maxn=1)
GM_EARTH = gm_earth_pre[0]

# Compute the SOI radius of the Earth
SOI_EARTH_R = ONE_AU * (GM_EARTH/GM_SUN) ** (2.0/5.0)

# Set one Lunar Distance (LD) in km (value from spaceweather.com)
ONE_LD = 384401.0

print(f'SOI of the Earth in LD: {SOI_EARTH_R/ONE_LD}')

SOI of the Earth in LD: 2.4054224328225597


In [4]:
# Let's obtain the orbit elements data of 2021 WP from 
# https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=2021%20WP

# Before we compute a state vector of the asteroid and the current distance
# to our home planet we need to define a function to round the data. A common
# convention for scientific work is to round the data to one significant
# digit. We create a lambda function that rounds the values based on the
# provided measurement error
round_sig = lambda value, err: np.round(value, -1*(int(np.floor(np.log10(err)))))

# Set now the perihelion in km
neo_2021wp_perihelion_km = spiceypy.convrt(round_sig(0.9320456652320572, \
                                                     4.1154E-5), \
                                            inunit='AU', outunit='km')

# Set the eccentricity
neo_2021wp_ecc = round_sig(0.1566053963605083, 0.00010949)

# Set the inclination, longitude of ascending node and argument of periapsis
# in radians
neo_2021wp_inc_rad = np.radians(round_sig(3.808107572352944, 0.0027162))
neo_2021wp_lnode_rad = np.radians(round_sig(58.20287162913875, 0.0021186))
neo_2021wp_argp_rad = np.radians(round_sig(306.7404738426862, 0.0045684))

# Set the mean anomaly and corresponding epoch in Julian Date (JD)
neo_2021wp_m0_at_t0_rad = np.radians(round_sig(91.92206685260463, 0.013333))
neo_2021wp_t0 = spiceypy.utc2et('2459600.5 JD')

# Set the orbital elements array
neo_2021wp_orbital_elements = [neo_2021wp_perihelion_km, \
                               neo_2021wp_ecc, \
                               neo_2021wp_inc_rad, \
                               neo_2021wp_lnode_rad, \
                               neo_2021wp_argp_rad, \
                               neo_2021wp_m0_at_t0_rad, \
                               neo_2021wp_t0, \
                               GM_SUN]

In [5]:
# Let's compute the first SOI crossing of the NEO (from interplanetary space to Earth's vicinity)

# First we set a counter for a while condition. When the NEO is entering the SOI, we increase the
# counter to leave the while condition
soi_crossing_cnt = 0

while soi_crossing_cnt == 0:

    # Increase ET time by 1 minute
    datetime_et += 60 # seconds

    # Compute the state vector of the NEO and Earth
    neo_2021wp_state_vector = spiceypy.conics(neo_2021wp_orbital_elements, datetime_et)
    earth_state_vector, _ = spiceypy.spkgeo(targ=399, \
                                            et=datetime_et, \
                                            ref='ECLIPJ2000',
                                            obs=10)

    # Compute the state vector of the NEO w.r.t. our home planet
    neo_2021wp_wrt_earth_state_vector = neo_2021wp_state_vector - earth_state_vector

    # Compute the current distance of the Earth and the NEO in km
    earth_2021wp_dist = spiceypy.vnorm(neo_2021wp_wrt_earth_state_vector[:3])

    # Check if the SOI is crossed. Increase counter and print some results
    if earth_2021wp_dist <= SOI_EARTH_R:
        soi_crossing_cnt += 1
        
        print(f"1st SOI crossing at UTC: " \
              f"{spiceypy.et2datetime(datetime_et).strftime(datetime_format)}")
        print(f"Intersection check in LD: {earth_2021wp_dist / ONE_LD}")


1st SOI crossing at UTC: 2021-11-19T03:32:59
Intersection check in LD: 2.405386171012861


In [6]:
# Now let's take a look at the orbital elements of the NEO in Earth-centric ECLIPJ2000 coordiantes
neo_elements_wrt_earth = spiceypy.oscelt(state=neo_2021wp_wrt_earth_state_vector,
                                         et=datetime_et,
                                         mu=GM_EARTH)

# Printing only the perigee and eccentricity.
# As we can see, that perigee is way larger than the radius of our planet. So there is no risk for
# an impact (the value is around 0.5 LD). The eccentricity indicates that the NEO is on a highly
# eccentric orbit. Thus, it will leave Earth's vicinity and the SOI.
print(f"Perigee of 2021 WP w.r.t. the Earth in km: {neo_elements_wrt_earth[0]}")
print(f"Eccentricity of 2021 WP w.r.t. the Earth: {neo_elements_wrt_earth[1]}")

Perigee of 2021 WP w.r.t. the Earth in km: 152774.92214815144
Eccentricity of 2021 WP w.r.t. the Earth: 8.907534305396858


In [7]:
# Now we compute the trajectory of the NEO within Earth's SOI in Earth-centric orbital elements
while soi_crossing_cnt == 1:

    # Increase ET time by 1 minute
    datetime_et += 60 # seconds

    # Compute the state vector of the NEO in Earth centric coordiantes
    neo_2021wp_wrt_earth_state_vector = spiceypy.conics(neo_elements_wrt_earth, datetime_et)

    # Compute the current distance of the Earth and the asteroid in km
    earth_2021wp_dist = spiceypy.vnorm(neo_2021wp_wrt_earth_state_vector[:3])

    # Check if the SOI is crossed. Increase counter and print some results
    if earth_2021wp_dist >= SOI_EARTH_R:
        soi_crossing_cnt += 1
        
        print(f"2nd SOI crossing at UTC: " \
              f"{spiceypy.et2datetime(datetime_et).strftime(datetime_format)}")
        print(f"Intersection check in LD: {earth_2021wp_dist / ONE_LD}")

2nd SOI crossing at UTC: 2021-11-23T15:25:59
Intersection check in LD: 2.405510448418613


In [8]:
# 2021 WP left the SOI. We can now compute Earth's state vector at the SOI-crossing-time and re-
# compute the state vector of the NEO in helio-centric coordiantes
earth_state_vector, _ = spiceypy.spkgeo(targ=399, \
                                        et=datetime_et, \
                                        ref='ECLIPJ2000',
                                        obs=10)

# Compute the state vector of the NEO in helio-centric coordiantes (correcting by the Earth's
# state vector)
neo_2021wp_state_vector = earth_state_vector + neo_2021wp_wrt_earth_state_vector

# Compute now the orbital elements, based on the newly computed NEO state vector.
# Did the orbital elements change at all?
neo_2021wp_orbital_elements_altered = spiceypy.oscelt(state=neo_2021wp_state_vector,
                                                      et=datetime_et,
                                                      mu=GM_SUN)

In [9]:
# Print some orbital elements:
#     - Before entering Earth's SOI
#     - After leaving Earth's SOI
#     - And the corresponding delta
#
# Note: we are not rounding any values here. Create your function to round the newly computed values
# based on the previous precision.
print(f"Perihelion in AU before and after (+ delta): " \
      f"{neo_2021wp_orbital_elements[0] / ONE_AU}, "
      f"{neo_2021wp_orbital_elements_altered[0] / ONE_AU}, "
      f"{(neo_2021wp_orbital_elements[0] - neo_2021wp_orbital_elements_altered[0]) / ONE_AU}")

print(f"Eccentricity before and after (+ delta) in AU: " \
      f"{neo_2021wp_orbital_elements[1]}, "
      f"{neo_2021wp_orbital_elements_altered[1]}, "
      f"{neo_2021wp_orbital_elements[1] - neo_2021wp_orbital_elements_altered[1]}")

print(f"Inclination in degrees before and after (+ delta): " \
      f"{np.degrees(neo_2021wp_orbital_elements[2])}, "
      f"{np.degrees(neo_2021wp_orbital_elements_altered[2])}, "
      f"{np.degrees(neo_2021wp_orbital_elements[2] - neo_2021wp_orbital_elements_altered[2])}")

print(f"Long. of. asc. node in degrees before and after (+ delta): " \
      f"{np.degrees(neo_2021wp_orbital_elements[3])}, "
      f"{np.degrees(neo_2021wp_orbital_elements_altered[3])}, "
      f"{np.degrees(neo_2021wp_orbital_elements[3] - neo_2021wp_orbital_elements_altered[3])}")

print(f"Arg. of. perihelion in degrees before and after (+ delta): " \
      f"{np.degrees(neo_2021wp_orbital_elements[4])}, "
      f"{np.degrees(neo_2021wp_orbital_elements_altered[4])}, "
      f"{np.degrees(neo_2021wp_orbital_elements[4] - neo_2021wp_orbital_elements_altered[4])}")

Perihelion in AU before and after (+ delta): 0.9320499999999998, 0.9185358482068057, 0.013514151793194198
Eccentricity before and after (+ delta) in AU: 0.1566, 0.16168752100441286, -0.0050875210044128705
Inclination in degrees before and after (+ delta): 3.808, 2.106572488148098, 1.7014275118519018
Long. of. asc. node in degrees before and after (+ delta): 58.202999999999996, 57.369864198134934, 0.8331358018650636
Arg. of. perihelion in degrees before and after (+ delta): 306.74, 301.4696164686528, 5.2703835313471705
